<a href="https://colab.research.google.com/github/amazuzu/hackaton_2021/blob/master/day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

day 2 run

In [ ]:
%unload_ext google.colab.data_table

The google.colab.data_table extension is not loaded.


In [ ]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [ ]:
print(twenty_train.target_names)
print(twenty_test.data[1])
for i,a in enumerate(twenty_train.target_names):
   print(f'{a} is {i}')

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: Rick Miller <rick@ee.uwm.edu>
Subject: X-Face?
Organization: Just me.
Lines: 17
Distribution: world
NNTP-Posting-Host: 129.89.2.33
Summary: Go ahead... swamp me.  <EEP!>

I'm not familiar at all with the format of these "X-Face:" thingies, but
after seeing them in some folks' headers, I've *got* to *see* them (and
maybe make one of my own)!

I've got "dpg-view" on my Linux box (which displays "uncompressed X-Faces")
and I've managed to compile [un]compface too... but now that I'm *looking*
for them, I can't seem to find any X-Face:'s in anyones news headers!  :-(

Could you, would you, please 

extracting features from text files

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(11314, 130107)

In [ ]:
# X_train_counts

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 130107)

running ML algorithms

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
print(clf)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)


make via pipeline


now lets measure performance

In [ ]:
from sklearn.pipeline import Pipeline
import numpy as np

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)

predicted = text_clf.predict(twenty_test.data)
np.mean(predicted == twenty_test.target)

0.7738980350504514

0.7738980350504514

In [ ]:
print(text_clf)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)


In [ ]:
print(twenty_train.data[14])

From: jllee@acsu.buffalo.edu (Johnny L Lee)
Subject: RE:  == MOVING SALE ===
Summary: RE:  === MOVING SALE ===
Organization: UB
Lines: 44
Nntp-Posting-Host: lictor.acsu.buffalo.edu

Reduced Prices! 
I have a list of things forsale on behalf of my brother, who's moving (moved
already)

								Offer:
1) Black and Decker Duster Plus (Portable Hand Vaccum)	
 	purchased for $32, 					  $12

2) SR-1000 Dual Cassette Portable Player, AM/FM
5-Band graphics Equalizer, high speed dubing, Duo 
Tape.Tape deck A, seems to have lost treble sound. 
But, I bet  it's fixable.
	purchased for $80					  $25

3)Monolux Zoom MicroScope, up to 1200X magnification
Made in Japan, includes case and accessories
	purchased for $50					  $20

4)Sunbeam 1400 Hair Dryer, the dryer you put your 
head under/into. You know, the ones you see in the salons.
(Don't ask me why my bro had it)
	purchased for $60				          $24

5)Everylast Speed Bag, all leather. Brand new, never 
used								  $10

6)Osterizer Pusle Mati

In [ ]:
print(twenty_train.target_names[predicted[14]])

soc.religion.christian


II Support Vector Machines

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
...                      ('tfidf', TfidfTransformer()),
...                      ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',
...                                            alpha=1e-3, n_iter_no_change=5, random_state=42)),
... ])

In [ ]:
 _ = text_clf_svm.fit(twenty_train.data, twenty_train.target)

In [ ]:
predicted_svm = text_clf_svm.predict(twenty_test.data)

In [ ]:
np.mean(predicted_svm == twenty_test.target)

0.8240839086563994

In [ ]:
print(predicted_svm[23])

14


**Grid Search**

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
               'tfidf__use_idf': (True, False),
               'clf__alpha': (1e-2, 1e-3),}
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)

/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


NameError: ignored

In [ ]:
print(gs_clf.best_score_)
print(gs_clf.best_params_)

0.9157684864695698
{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


Try RandomForestClassifier

